In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics.engine.results import Results
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [2]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
    "path" : str(DATASETS_DIR.absolute()),
    'train': str(TRAIN_IMAGES_DIR.absolute()),
    'val': str(VAL_IMAGES_DIR.absolute()),
    'test': str(TEST_IMAGES_DIR.absolute()),
    'nc': num_classes,
    'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [3]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("zindi_challenge_cacao/train*/weights/best.pt"))[-1]
# BEST_PATH = "runs/detect/train3/weights/best.pt"
BEST_PATH

'zindi_challenge_cacao/train/weights/best.pt'

In [4]:
for flag, v in ExifTags.TAGS.items():
    if v == "Orientation":
        break


def load_image_(filepath):
    image = Image.open(filepath)
    # return image

    exif = image._getexif()
    if exif is None:
        return image

    orientation_value = exif.get(flag, None)

    if orientation_value == 3:
        image = image.rotate(180, expand=True)
    elif orientation_value == 6:
        image = image.rotate(270, expand=True)
    elif orientation_value == 8:
        image = image.rotate(90, expand=True)
    return image

from ultralytics.utils.patches import imread
import cv2

def load_image(filepath):
    return imread(filepath, cv2.IMREAD_COLOR)


flag

274

In [5]:
# Validate the model on the validation set
BEST_CFG = sorted(glob("zindi_challenge_cacao/train*/args.yaml"))[-1]
BEST_CFG

'zindi_challenge_cacao/train/args.yaml'

In [6]:
import yaml
with open(BEST_CFG, 'r') as f:
	cfg: dict = yaml.safe_load(f)
	print(cfg)

{'task': 'detect', 'mode': 'train', 'model': 'yolo11l.pt', 'data': 'data.yaml', 'epochs': 200, 'time': None, 'patience': 20, 'batch': 16, 'imgsz': 1024, 'save': True, 'save_period': -1, 'cache': False, 'device': [0, 1], 'workers': 8, 'project': 'zindi_challenge_cacao', 'name': 'train', 'exist_ok': False, 'pretrained': True, 'optimizer': 'auto', 'verbose': True, 'seed': 0, 'deterministic': True, 'single_cls': False, 'rect': False, 'cos_lr': False, 'close_mosaic': 10, 'resume': False, 'amp': True, 'fraction': 1.0, 'profile': False, 'freeze': None, 'multi_scale': True, 'overlap_mask': True, 'mask_ratio': 4, 'dropout': 0.3, 'val': True, 'split': 'val', 'save_json': False, 'conf': None, 'iou': 0.42361207118859606, 'max_det': 150, 'half': True, 'dnn': False, 'plots': True, 'source': None, 'vid_stride': 1, 'stream_buffer': False, 'visualize': False, 'augment': True, 'agnostic_nms': False, 'classes': None, 'retina_masks': False, 'embed': None, 'show': False, 'save_frames': False, 'save_txt': F

In [7]:
# Batch size for predictions
batch_size = 16

cfg["device"] = "cuda"
cfg["batch"] = batch_size
cfg["conf"] = 0.
cfg["verbose"] = False

cfg.pop("source")
# cfg.pop("batch_size")
cfg.pop("visualize")

keys = list(cfg.keys())
for col in keys:
	if "show" in col or "save" in col:
		cfg.pop(col)

In [8]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

# Path to the test images directory
test_dir_path = TEST_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []

# Batch size for predictions
batch_size = 16

# Process images in batches
for i in tqdm(range(0, len(image_files), batch_size)):
	batch_files = image_files[i:i + batch_size]
	batch_images = [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

	# Make predictions on the batch of images
	results = model.predict(
		batch_images,
		**cfg,
	)

	# Iterate through each result in the batch
	for img_file, result in zip(batch_files, results):
		boxes = result.boxes.xyxy.tolist() if result.boxes else []  # Bounding boxes in xyxy format
		classes = result.boxes.cls.tolist() if result.boxes else []  # Class indices
		confidences = result.boxes.conf.tolist() if result.boxes else []  # Confidence scores
		names = result.names  # Class names dictionary

		if boxes:  # If detections are found
			for box, cls, conf in zip(boxes, classes, confidences):
				x1, y1, x2, y2 = box
				detected_class = names[int(cls)]  # Get the class name from the names dictionary

				# Add the result to the all_data list
				all_data.append(
					{
						"Image_ID": str(img_file),
						"class": detected_class,
						"confidence": conf,
						"ymin": y1,
						"xmin": x1,
						"ymax": y2,
						"xmax": x2,
					}
				)
		else:  # If no objects are detected
			all_data.append(
				{
					"Image_ID": str(img_file),
					"class": "None",
					"confidence": None,
					"ymin": None,
					"xmin": None,
					"ymax": None,
					"xmax": None,
				}
			)


100%|██████████| 102/102 [02:09<00:00,  1.27s/it]


In [9]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [10]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_cWEAQI.jpeg,healthy,0.673828,5.859375,0.976562,3970.702881,1664.062378
1,ID_cWEAQI.jpeg,healthy,0.108582,329.833954,1316.406128,1249.023315,1800.000000
2,ID_cWEAQI.jpeg,healthy,0.100891,177.734360,0.000000,1560.546753,496.093719
3,ID_cWEAQI.jpeg,anthracnose,0.052246,2710.937256,610.351501,3976.562256,1496.093628
4,ID_cWEAQI.jpeg,anthracnose,0.006123,3234.374756,864.257751,3828.124756,1316.406128


In [11]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,2.439000e+05,243900.000000,243900.000000,243900.000000,243900.000000
mean,9.549118e-03,928.759113,751.294717,1356.227478,1181.338937
std,7.536789e-02,1018.303390,869.350029,1154.302567,996.690944
min,6.556511e-07,0.000000,0.000000,0.000000,0.000000
25%,1.138449e-05,31.585936,20.000000,406.249969,397.460907
50%,3.427267e-05,603.808594,449.000000,1063.789062,920.390625
75%,1.488924e-04,1394.812500,1095.609253,2048.000000,1798.828003
max,9.238281e-01,4057.453125,4059.468750,4128.000000,4128.000000


In [12]:
sub['class'].value_counts()

class
healthy        99348
cssvd          75363
anthracnose    69189
Name: count, dtype: int64

In [13]:
sub.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64

class
healthy        1153
cssvd           801
anthracnose     694
None             57
Name: count, dtype: int6

In [14]:
sub.to_csv("dataset/predictions/06-2-predictions.csv", index=False)

In [15]:
sub["confidence"].describe()

count    2.439000e+05
mean     9.549118e-03
std      7.536789e-02
min      6.556511e-07
25%      1.138449e-05
50%      3.427267e-05
75%      1.488924e-04
max      9.238281e-01
Name: confidence, dtype: float64

In [16]:
import pandas as pd

sub = pd.read_csv('dataset/predictions/06-2-predictions.csv')

sub.sample(6)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
211193,ID_fydp9k.jpg,healthy,0.000002,695.625000,0.000000,806.875000,2.773438
171583,ID_UDljRB.jpeg,anthracnose,0.000003,2775.937256,2732.624756,3405.937256,3024.000000
206433,ID_qfixBm.jpg,healthy,0.000349,1686.187500,1861.500000,2208.937500,2310.937500
205353,ID_yEGJej.jpg,cssvd,0.299805,0.000000,6.375000,797.273438,1093.312500
77044,ID_deS0wq.jpg,cssvd,0.000150,2081.437500,1839.187500,2613.750000,2291.812500
69167,ID_ydmVnV.jpg,cssvd,0.000283,0.410156,783.125000,41.796875,960.000000


In [17]:
sub["Image_ID"].value_counts().describe()

count    1626.0
mean      150.0
std         0.0
min       150.0
25%       150.0
50%       150.0
75%       150.0
max       150.0
Name: count, dtype: float64

In [18]:
sub["Image_ID"].nunique()

1626

In [19]:
sub.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64